In [ ]:
!pip install huggingface_hub[hf_xet] catboost gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 118.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 67.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import string

from tqdm.auto import tqdm
from transformers import pipeline
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

tqdm.pandas()

In [ ]:
original_data = pd.read_csv("./drive/MyDrive/paragraphs.csv")

In [ ]:
data = original_data.copy()

In [ ]:
data.text = data.text.progress_apply(lambda x: x.replace("\n", " ").lower())

  0%|          | 0/1701 [00:00<?, ?it/s]

In [ ]:
clf = pipeline("zero-shot-classification",
               model="facebook/bart-large-mnli",
              device_map="cuda",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
text = """2 Marquis de Custine, Journey for Our Time: The Journals of Marquis de Custine, ed. and trans.
 Phyllis Penn Kohler (1843; New York: Pellegrini and Cudahy, 1951); Baron August von
 Haxthausen, The Russian Empire: Its People, Institutions and Resources, 2 vols., trans. Robert
 Farie (1847; London: Chapman and Hall, 1856); Sir Donald Mackenzie Wallace, Russia
 on the Eve of War and Revolution, ed. and intro. Cyril E. Black (1877; New York: Random
 House, 1961); Alfred Rambaud, The History of Russia from the Earliest Times to 1 877, trans.
 Leonora B. Lang, 2 vols. (1878; New York: Hovendon Company, 1886); Anatole Leroy-
 Beaulieu, The Empire of the Tsars and the Russians, 3 vols., trans. Zénı̈ade A. Ragozin (New
 York: Knickerbocker Press, 1902); George F. Kennan, Siberia and the Exile System, 2 vols.
 (New York: Century, 1891).
 3 David C. Engerman, Modernization from the Other Shore: American Intellectuals and the
 Romance of Russian Development (Cambridge, Mass.: Harvard University Press, 2003),
 pp. 28–53."""

hypo = "this historical text refers to {}"

candidate_labels = [
    "sources, authors, questions, links",
    "other"
]

clf(
    text,
    candidate_labels=candidate_labels,
    hypothesis_template=hypo,
    multi_label=False
)

{'sequence': '2 Marquis de Custine, Journey for Our Time: The Journals of Marquis de Custine, ed. and trans.\n Phyllis Penn Kohler (1843; New York: Pellegrini and Cudahy, 1951); Baron August von\n Haxthausen, The Russian Empire: Its People, Institutions and Resources, 2 vols., trans. Robert\n Farie (1847; London: Chapman and Hall, 1856); Sir Donald Mackenzie Wallace, Russia\n on the Eve of War and Revolution, ed. and intro. Cyril E. Black (1877; New York: Random\n House, 1961); Alfred Rambaud, The History of Russia from the Earliest Times to 1 877, trans.\n Leonora B. Lang, 2 vols. (1878; New York: Hovendon Company, 1886); Anatole Leroy-\n Beaulieu, The Empire of the Tsars and the Russians, 3 vols., trans. Zénı̈ade A. Ragozin (New\n York: Knickerbocker Press, 1902); George F. Kennan, Siberia and the Exile System, 2 vols.\n (New York: Century, 1891).\n 3 David C. Engerman, Modernization from the Other Shore: American Intellectuals and the\n Romance of Russian Development (Cambridge, Mas

In [ ]:
result = pd.DataFrame(columns=["text", "label", "score"])

for i, row in tqdm(data.iterrows(), total=data.shape[0]):
    preds = clf(
        row.text,
        candidate_labels=candidate_labels,
        hypothesis_template=hypo,
        multi_label=False
    )
    result.loc[i] = [
        row.text,
        preds["labels"],
        preds["scores"]
    ]

  0%|          | 0/1701 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
result.loc[
    (result.label.apply(lambda x: x[0] != "other")) & \
     (result.score.apply(lambda x: x[0] > 0.65))
    ].to_csv("./drive/MyDrive/saq.csv")

In [227]:
result.loc[
    (result.label.apply(lambda x: x[0] == "other")) & \
     (result.score.apply(lambda x: x[0] > 0.7))
    ][["text"]].drop_duplicates(subset="text").to_csv("./drive/MyDrive/raw7wd.csv")

In [228]:
fixed_authors = pd.read_csv("./drive/MyDrive/saq065.csv")
fixed_authors["label"] = "author"

In [229]:
raw_data_fixed = pd.read_csv("./drive/MyDrive/raw_fixed.csv")
raw_data_fixed["label"] = "other"

In [230]:
raw_data_fixed = raw_data_fixed.drop_duplicates(subset="text")

In [231]:
raw_data_fixed = raw_data_fixed.drop("Unnamed: 0", axis=1)

In [232]:
fixed_authors = fixed_authors[["text", "label"]]

In [233]:
data = pd.concat([fixed_authors, raw_data_fixed])

In [234]:
data = data.sample(frac=1)
data.reset_index(drop=True, inplace=True)

In [235]:
data["len"] = data.text.apply(len)

In [236]:
data["punct_count"] = data.text.apply(lambda x: len([c for c in x if c in string.punctuation]))

In [237]:
data["numbers_count"] = data.text.apply(lambda x: len([c for c in x if c.isdigit()]))

In [238]:
data["word_numbers_count"] = data.text.apply(lambda x: len([w for w in x.split() if w.isdigit()]))

In [239]:
def process_text(text):
    return "".join([char for char in text.lower() if char.isalnum() or char.isspace()]).strip()

In [240]:
data.text = data.text.apply(process_text)

In [241]:
data["len_after_prc"] = data.text.apply(len)

In [242]:
columns = [
    "len",
    "punct_count",
    "numbers_count",
    "word_numbers_count",
    # ------------------
    "len_after_prc",
]

In [243]:
vectorizer = TfidfVectorizer(
    max_df=0.9,
    min_df=5,
    ngram_range=(1,2),
    norm='l2'
)

In [244]:
vectorizer.fit(original_data.text.apply(process_text))

TfidfVectorizer(max_df=0.9, min_df=5, ngram_range=(1, 2))

In [245]:
tfidf_matrix = vectorizer.transform(data.text)

In [246]:
tfidf_matrix = tfidf_matrix.toarray()

In [247]:
full_data = np.concatenate(
    [
        tfidf_matrix,
        data[columns].to_numpy()
    ], axis=1)

In [248]:
X_train, X_test, y_train, y_test = train_test_split(full_data, data.label, test_size=0.1, stratify=data.label)

In [249]:
model = CatBoostClassifier()

In [250]:
model.fit(X_train, y_train, verbose=100)

Learning rate set to 0.006636
0:	learn: 0.6870335	total: 32ms	remaining: 32s
100:	learn: 0.3060323	total: 2.67s	remaining: 23.8s
200:	learn: 0.1807018	total: 5.34s	remaining: 21.2s
300:	learn: 0.1324924	total: 7.9s	remaining: 18.3s
400:	learn: 0.1144381	total: 10.4s	remaining: 15.6s
500:	learn: 0.0984148	total: 13s	remaining: 12.9s
600:	learn: 0.0855965	total: 15.6s	remaining: 10.4s
700:	learn: 0.0760084	total: 18.3s	remaining: 7.81s
800:	learn: 0.0679945	total: 21.3s	remaining: 5.28s
900:	learn: 0.0600853	total: 24.1s	remaining: 2.65s
999:	learn: 0.0526992	total: 27s	remaining: 0us


In [251]:
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

      author       0.96      0.93      0.94        27
       other       0.86      0.92      0.89        13

    accuracy                           0.93        40
   macro avg       0.91      0.92      0.92        40
weighted avg       0.93      0.93      0.93        40



In [252]:
model.fit(full_data, data.label, verbose=100)

Learning rate set to 0.006944
0:	learn: 0.6866911	total: 44.8ms	remaining: 44.8s
100:	learn: 0.3004421	total: 2.68s	remaining: 23.8s
200:	learn: 0.1767941	total: 5.3s	remaining: 21.1s
300:	learn: 0.1306423	total: 8.03s	remaining: 18.6s
400:	learn: 0.1123368	total: 10.8s	remaining: 16.1s
500:	learn: 0.0971422	total: 13.4s	remaining: 13.3s
600:	learn: 0.0842588	total: 16s	remaining: 10.6s
700:	learn: 0.0745196	total: 18.6s	remaining: 7.94s
800:	learn: 0.0644141	total: 21.4s	remaining: 5.31s
900:	learn: 0.0570043	total: 24.1s	remaining: 2.65s
999:	learn: 0.0497837	total: 26.7s	remaining: 0us


In [254]:
data_processed = original_data.copy()

In [255]:
data_processed["len"] = data_processed.text.apply(len)
data_processed["punct_count"] = data_processed.text.apply(lambda x: len([c for c in x if c in string.punctuation]))
data_processed["numbers_count"] = data_processed.text.apply(lambda x: len([c for c in x if c.isdigit()]))
data_processed["word_numbers_count"] = data_processed.text.apply(lambda x: len([w for w in x.split() if w.isdigit()]))
data_processed.text = data_processed.text.apply(process_text)
data_processed["len_after_prc"] = data_processed.text.apply(len)

In [256]:
data_processed_tfidf = vectorizer.transform(data_processed.text)

In [257]:
data_processed_tfidf = data_processed_tfidf.toarray()

In [258]:
test = np.concatenate(
    [
        data_processed_tfidf,
        data_processed[columns].to_numpy()
    ], axis=1)

In [259]:
preds = model.predict(test)

array(['other', 'other', 'other', ..., 'author', 'author', 'other'],
      dtype=object)

In [261]:
test_data = original_data.copy()
test_data["label"] = preds

In [263]:
test_data.to_csv("./drive/MyDrive/processed_data.csv", index=False)

In [266]:
test_data.loc[test_data.label == "other"].to_csv("./drive/MyDrive/raw_data3.csv", index=False)